# DRL usage example

## Objectives

1. Determine if energy is saved when using a smart Agent vs fixed set-point schedule.
2. Use existing agents, but we should control what observation variables are being used.
* Zone Temperature and Humidity
* Outdoor Temperature and Humidity
* Occupancy schedule
* Zone HVAC set-point



Additional data that we can include later in addition to the current observation values:
* min/max/mean over the last 15 minutes. 
* Forecast for the next 15 minutes (weather and occupancy).


### Baseline measurement
OpenStudio BLC Office Medium ASHRAE 2016 without modifications

In this notebook example, Stable Baselines 3 has been used to train and to load an agent. However, Sinergym is completely agnostic to any DRL algorithm (although there are custom callbacks for SB3 specifically) and can be used with any DRL library that works with gymnasium environments.

## Training a model

We are going to rely on the script available in the repository root called `DRL_battery.py`. This script applies all the possibilities that Sinergym has to work with deep reinforcement learning algorithms and set parameters to everything so that we can define the training options from the execution of the script easily by a JSON file.

For more information about how run `DRL_battery.py`, please, see [Train a model](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#train-a-model).

In [1]:
import sys
from datetime import datetime

import gymnasium as gym
import numpy as np
import wandb
from stable_baselines3 import *
from stable_baselines3.common.callbacks import CallbackList
from stable_baselines3.common.logger import HumanOutputFormat, Logger
from stable_baselines3.common.monitor import Monitor

import sinergym
import sinergym.utils.gcloud as gcloud
from sinergym.utils.callbacks import *
from sinergym.utils.constants import *
from sinergym.utils.logger import CSVLogger, WandBOutputFormat
from sinergym.utils.rewards import *
from sinergym.utils.wrappers import *

First let's define some variables for the execution.

In [2]:
# Environment ID
environment = "Eplus-office-mixed-discrete-stochastic-v1"
# Training episodes
episodes = 200
#Name of the experiment
experiment_date = datetime.today().strftime('%Y-%m-%d_%H:%M')
experiment_name = 'SB3_DQN-' + environment + \
    '-episodes-' + str(episodes)
experiment_name += '_' + experiment_date

We can combine this experiment executions with [Weights&Biases](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#weights-and-biases-structure) in order to host all information extracted. With *wandb*, it’s possible to track and visualize all DRL training process in real time, register hyperparameters and details of each experiment, save artifacts such as models and *sinergym* output, and compare between different executions.

In [3]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
# Create wandb.config object in order to log all experiment params
experiment_params = {
    'sinergym-version': sinergym.__version__,
    'python-version': sys.version
}
experiment_params.update({'environment':environment,
                          'episodes':episodes,
                          'algorithm':'SB3_DQN'})

# Get wandb init params (you have to specify your own project and entity)
wandb_params = {"project": 'zf_test',
                "entity": 'zephframe'}
# Init wandb entry
run = wandb.init(
    name=experiment_name + '_' + wandb.util.generate_id(),
    config=experiment_params,
    ** wandb_params
)

wandb: Currently logged in as: zephframe. Use `wandb login --relogin` to force relogin


Now we are ready to create the Gymnasium Environment. Here we use the environment name defined, remember that you can [change default environment configuration](https://ugr-sail.github.io/sinergym/compilation/main/pages/notebooks/change_environment.html#Changing-an-environment-registered-in-Sinergym). We will create a eval_env too in order to interact in the evaluation episodes. We can overwrite the env name with experiment name if we want.

In [4]:
env = gym.make(environment, env_name=experiment_name)
eval_env = gym.make(environment, env_name=experiment_name+'_EVALUATION')

[2023-05-12 04:17:27,838] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-05-12 04:17:27,839] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-05-12 04:17:27,842] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-05-12 04:17:27,846] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-05-12 04:17:27,847] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:Setting up action definitio

We can also add a Wrapper to the environment, we are going to use a Logger (extension of `gym.Wrapper`) this is used to monitor and log the interactions with the environment and save the data into a CSV. Files generated will be stored as artifact in *wandb* too.

In [5]:
env = LoggerWrapper(env)

At this point, we have the environment set up and ready to be used. We are going to create our learning model (Stable Baselines 3 DQN), but we can use any other algorithm.

In [6]:
model = DQN('MlpPolicy', env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Now we need to calculate the number of timesteps of each episode for the evaluation. Evaluation will execute the current model during a number of episodes determined to decide if it is the best current version of the model at that point of the training. Output generated will be stored in *wandb* server too.

In [7]:
n_timesteps_episode = env.simulator._eplus_one_epi_len / \
                      env.simulator._eplus_run_stepsize


We are going to use the LoggerEval callback to print and save the best model evaluated during training.

In [8]:
callbacks = []

# Set up Evaluation and saving best model
eval_callback = LoggerEvalCallback(
    eval_env,
    best_model_save_path=eval_env.simulator._env_working_dir_parent +
    '/best_model/',
    log_path=eval_env.simulator._env_working_dir_parent +
    '/best_model/',
    eval_freq=n_timesteps_episode * 2,
    deterministic=True,
    render=False,
    n_eval_episodes=1)
callbacks.append(eval_callback)

callback = CallbackList(callbacks)

In order to track all the training process in *wandb*, it is necessary to create a callback with a compatible wandb output format (which call *wandb* log method in the learning algorithm process).

In [9]:
# wandb logger and setting in SB3
logger = Logger(
    folder=None,
    output_formats=[
        HumanOutputFormat(
            sys.stdout,
            max_length=120),
        WandBOutputFormat()])
model.set_logger(logger)
# Append callback
log_callback = LoggerCallback()
callbacks.append(log_callback)


callback = CallbackList(callbacks)

This is the number of total time steps for the training.

In [10]:
timesteps = episodes * n_timesteps_episode

Now is time to train the model with the callbacks defined earlier. This may take a few minutes, depending on your computer.

In [11]:
model.learn(
    total_timesteps=timesteps,
    callback=callback,
    log_interval=1)

[2023-05-12 04:17:51,877] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-12 04:17:52,108] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:EnergyPlus working directory is in /root/my-project/examples/Eplus-env-SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17-res1/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-05-12 04:19:40,144] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-12 04:19:40,146] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-12 04:19:40,384] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:EnergyPlus working directory is in /root/my-project/examples/Eplus-env-SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17-res1/Eplus-env-sub_run2


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


-------------------------------------------------------------------------------
| action/                                                        |            |
|    Office_Cooling_RL                                           | 25.5       |
|    Office_Heating_RL                                           | 19.1       |
| action_simulation/                                             |            |
|    Office_Cooling_RL                                           | 25.5       |
|    Office_Heating_RL                                           | 19.1       |
| episode/                                                       |            |
|    comfort_violation_time(%)                                   | 8.03       |
|    cumulative_comfort_penalty                                  | -3.59e+03  |
|    cumulative_power                                            | 7.03e+08   |
|    cumulative_power_penalty                                    | -7.03e+04  |
|    cumulative_reward                  

...

[2023-05-12 15:41:24,572] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_EVALUATION_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-12 15:41:24,573] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_EVALUATION_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-12 15:41:24,930] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_EVALUATION_MainThread_ROOT INFO:EnergyPlus working directory is in /root/my-project/examples/Eplus-env-SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_EVALUATION-res1/Eplus-env-sub_run199


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-05-12 15:43:16,035] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_EVALUATION_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-05-12 15:43:16,036] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_EVALUATION_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-05-12 15:43:16,389] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_EVALUATION_MainThread_ROOT INFO:EnergyPlus working directory is in /root/my-project/examples/Eplus-env-SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_EVALUATION-res1/Eplus-env-sub_run200


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


Eval num_timesteps=7008000, episode_reward=-28466.00 +/- 0.00
Episode length: 35040.00 +/- 0.00
--------------------------------------------------------------------------------
| action/                                                        |             |
|    Office_Cooling_RL                                           | 26.8        |
|    Office_Heating_RL                                           | 18          |
| action_simulation/                                             |             |
|    Office_Cooling_RL                                           | 26.8        |
|    Office_Heating_RL                                           | 18          |
| episode/                                                       |             |
|    comfort_violation_time(%)                                   | 27.9        |
|    cumulative_comfort_penalty                                  | -1.35e+04   |
|    cumulative_power                                            | 4.36e+08    |
|    cumulati

Now, we save the current model (model version when training has finished).

In [12]:
model.save(env.simulator._env_working_dir_parent + '/' + experiment_name)

And as always, remember to close the environment.

In [13]:
env.close()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[2023-05-13 02:05:37,600] EPLUS_ENV_SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


We have to upload all Sinergym output as *wandb* artifact. This output include all sinergym_output (and LoggerWrapper CSV files) and models generated in training and evaluation episodes.

In [14]:
artifact = wandb.Artifact(
        name="training",
        type="experiment1")
artifact.add_dir(
        env.simulator._env_working_dir_parent,
        name='training_output/')
artifact.add_dir(
    eval_env.simulator._env_working_dir_parent,
    name='evaluation_output/')
run.log_artifact(artifact)

# wandb has finished
run.finish()

wandb: Adding directory to artifact (/root/my-project/examples/Eplus-env-SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17-res1)... Done. 2.1s
wandb: Adding directory to artifact (/root/my-project/examples/Eplus-env-SB3_DQN-Eplus-office-mixed-discrete-stochastic-v1-episodes-200_2023-05-12_04:17_EVALUATION-res1)... Done. 1.6s


wandb: Network error (TransientError), entering retry loop.


action/Office_Cooling_RL,▂▁▄▂▆▁▆▄▆▂▆▄▄▅▁▆▃▇▆▄▃▆▇█▄▅▇▆▄▅▄██▅█▇▆▅▆▇
action/Office_Heating_RL,▇█▆█▃█▃▅▃▆▃▅▅▄█▃▇▃▄▅▇▃▂▁▅▄▂▃▅▃▄▁▁▄▁▂▃▄▄▂
action_simulation/Office_Cooling_RL,▂▁▄▂▆▁▆▄▆▂▆▄▄▅▁▆▃▇▆▄▃▆▇█▄▅▇▆▄▅▄██▅█▇▆▅▆▇
action_simulation/Office_Heating_RL,▇█▆█▃█▃▅▃▆▃▅▅▄█▃▇▃▄▅▇▃▂▁▅▄▂▃▅▃▄▁▁▄▁▂▃▄▄▂
episode/comfort_violation_time(%),▁▁▂▁▄▂▄▃▄▂▄▃▄▃▂▆▃▆▄▄▂▄▅█▃▄▇▄▃▅▃█▇▃▄▄▄▄▃▆
episode/cumulative_comfort_penalty,██▇█▄▇▅▇▇▇▆▇▆▇▆▄▇▄▇▆█▆▄▁▇▄▄▆▇▆█▃▄█▇▅▆▆▇▆
episode/cumulative_power,▇█▆█▃▇▃▅▃▆▃▅▄▄▆▃▅▂▃▄▅▃▂▁▄▄▂▂▄▄▅▁▁▄▂▂▃▃▃▂
episode/cumulative_power_penalty,▂▁▃▁▆▂▆▄▆▃▆▄▅▅▃▆▄▇▆▅▄▆▇█▅▅▇▇▅▅▄██▅▇▇▆▆▆▇
episode/cumulative_reward,▃▁▃▁▄▂▅▄▆▄▇▅▅▇▁▅▄▆▇▅▄▆▆▄▆▃▆█▆▅▆▆▆▇█▇▆▆█▇
episode/ep_length,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode/mean_comfort_penalty,██▇█▄▇▅▇▇▇▆▇▆▇▆▄▇▄▇▆█▆▄▁▇▄▄▆▇▆█▃▄█▇▅▆▆▇▆


We have all the experiments results in our local computer, but we can see the execution in *wandb* too:


- If we check our projects, we can see the execution allocated:

![wandb_projects1](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_projects1.png?raw=true)


- Hyperparameters tracked in the training experiment:

![wandb_training_hyperparameters](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_training_hyperparameters.png?raw=true)



- Artifacts registered (if evaluation is enabled, best model is registered too):
  
![wandb_training_artifact](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_training_artifact.png?raw=true)



- Visualization of metrics in real time:
  
![wandb_training_charts](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_training_charts.png?raw=true)

## Loading a model

We are going to rely on the script available in the repository root called `load_agent.py`. This script applies all the possibilities that Sinergym has to work with deep reinforcement learning models loaded and set parameters to everything so that we can define the load options from the execution of the script easily by a JSON file.

For more information about how run `load_agent.py`, please, see [Load a trained model](https://ugr-sail.github.io/sinergym/compilation/main/pages/deep-reinforcement-learning.html#load-a-trained-model).

First we define the Sinergym environment ID where we want to check the loaded agent and the name of the evaluation experiment.

In [16]:
# Environment ID
environment = "Eplus-office-mixed-discrete-stochastic-v1"
# Episodes
episodes=5
# Evaluation name
evaluation_date = datetime.today().strftime('%Y-%m-%d_%H:%M')
evaluation_name = 'SB3_DQN-EVAL-' + environment + \
    '-episodes-' + str(episodes)
evaluation_name += '_' + evaluation_date

We can also use *wandb* here. We can allocate this evaluation of a loaded model in other project in order to not merge experiments. 

In [17]:

# Create wandb.config object in order to log all experiment params
experiment_params = {
    'sinergym-version': sinergym.__version__,
    'python-version': sys.version
}
experiment_params.update({'environment':environment,
                          'episodes':episodes,
                          'algorithm':'SB3_DQN'})

# Get wandb init params (you have to specify your own project and entity)
wandb_params = {"project": 'sinergym_evaluations',
                "entity": 'alex_ugr'}
# Init wandb entry
run = wandb.init(
    name=experiment_name + '_' + wandb.util.generate_id(),
    config=experiment_params,
    ** wandb_params
)

We make the gymnasium environment and wrap with LoggerWrapper. We can use the evaluation experiment name to rename the environment.

In [20]:
env=gym.make(environment, env_name=evaluation_name)
env=LoggerWrapper(env)

[2023-04-05 15:12:10,652] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-04-05 15:12:10,654] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-04-05 15:12:10,659] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-04-05 15:12:10,663] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-04-05 15:12:10,664] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:Setting up action definition in building model if exists...


We load the Stable Baselines 3 DQN model using the model allocated in our local computer, although we can use a remote model allocated in *wandb* from other training experiment.

In [21]:
# get wandb artifact path (to load model)
load_artifact_entity = 'alex_ugr'
load_artifact_project = 'sinergym'
load_artifact_name = 'training'
load_artifact_tag = 'latest'
load_artifact_model_path = 'evaluation_output/best_model/model.zip'
wandb_path = load_artifact_entity + '/' + load_artifact_project + \
    '/' + load_artifact_name + ':' + load_artifact_tag
# Download artifact
artifact = run.use_artifact(wandb_path)
artifact.get_path(load_artifact_model_path).download('.')
# Set model path to local wandb file downloaded
model_path = './' + load_artifact_model_path
model = DQN.load(model_path)

As we can see, The *wandb* model we want to load can come from an artifact of an different entity or project from the one we are using to register the evaluation of the loaded model, as long as it is accessible.
The next step is use the model to predict actions and interact with the environment in order to collect data to evaluate the model.

In [22]:
for i in range(episodes):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a, _ = model.predict(obs)
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:
            current_month = info['month']
            print(info['month'], sum(rewards))
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

[2023-04-05 15:12:27,791] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-04-05 15:12:27,939] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09-res1/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


1 -1.086220480544118
2 -2079.9961147920526
3 -4054.5450027104594
4 -5315.115280552358
5 -6240.707702132893
6 -7147.282200268969
7 -9757.11094202821
8 -12816.141002414372
9 -15759.877949459922
10 -18249.178074198662
11 -19008.92750593897
12 -20059.684052467783
1 -22170.235119703077
Episode  0 Mean reward:  -0.6327121894892542 Cumulative reward:  -22170.235119703077
[2023-04-05 15:13:01,011] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-04-05 15:13:01,013] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-04-05 15:13:01,189] EPLUS_ENV_SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09-res1/Eplus-env-sub_run2
1 -1.086220480544118
2 -2077.808749644109
3 -4069.981

Finally, we register the evaluation data in wandb as an artifact to save it.

In [25]:
artifact = wandb.Artifact(
    name="evaluating",
    type="evaluation1")
artifact.add_dir(
    env.simulator._env_working_dir_parent,
    name='evaluation_output/')

run.log_artifact(artifact)

# wandb has finished
run.finish()

wandb: Adding directory to artifact (/workspaces/sinergym/examples/Eplus-env-SB3_DQN-EVAL-Eplus-demo-v1-episodes-5_2023-04-05_15:09-res1)... Done. 0.1s


We have the loaded model results in our local computer, but we can see the execution in *wandb* too:

- If we check the wandb project list, we can see that sinergym_evaluations project has a new run:

![wandb_project2](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_project2.png?raw=true)


- Hyperparameters tracked in the evaluation experiment and we can see the previous training artifact used to load the model:

![wandb_evaluating_hyperparameters](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_evaluating_hyperparameters.png?raw=true)



- Artifact registered with Sinergym Output (and CSV files generated with the Logger Wrapper):
  
![wandb_evaluating_artifact](https://github.com/ugr-sail/sinergym/blob/main/images/wandb_evaluating_artifact.png?raw=true)